<a href="https://colab.research.google.com/github/likhitha-kadiyala/Sai-Likhitha_INFO5731_Spring2021/blob/main/In_class_exercise/inclass9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
import pandas as pd
import nltk
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, KFold, cross_val_score
from xgboost import XGBClassifier

In [ ]:
from google.colab import files
loaded_files=files.upload ()

Saving stsa-test.txt to stsa-test.txt
Saving stsa-train.txt to stsa-train.txt


In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [ ]:
train_dataframe=pd.read_fwf("stsa-train.txt", header=None)
#train_dataframe= pd.DataFrame(my_data_train)
train_dataframe = train_dataframe.rename(columns={0: "Sentiment", 1: "Text"})
test_dataframe = pd.read_fwf("stsa-test.txt", header=None)
#test_dataframe =  pd.DataFrame(my_data_test)
test_dataframe = test_dataframe.rename(columns={0: "Sentiment", 1: "Text"})

In [ ]:
#Removal of special characters:

train_dataframe['Cleaned_Data'] = train_dataframe['Text'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

#Removal of punctuations :

train_dataframe['Cleaned_Data'] = train_dataframe['Cleaned_Data'].str.replace('[^\w\s]','')

train_dataframe['Cleaned_Data'] = train_dataframe['Cleaned_Data'].apply(lambda x :x.lower())

#Removing stopwords by using stopwordslist

stop = stopwords.words('english')
train_dataframe['Cleaned_Data'] = train_dataframe['Cleaned_Data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

from textblob import TextBlob
nltk.download('punkt')
train_dataframe['Cleaned_Data'] = train_dataframe['Cleaned_Data'].apply(lambda x: TextBlob(x).words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from textblob import Word

train_dataframe['Cleaned_Data'] = train_dataframe['Cleaned_Data'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
train_tfidf_vect = TfidfVectorizer(analyzer = 'word')
X_data = train_tfidf_vect.fit_transform(train_dataframe['Cleaned_Data'])

In [ ]:
x_train_data, x_validation_data, y_train_data, y_validation_data = train_test_split(X_data, train_dataframe['Sentiment'],test_size=0.2)


In [ ]:
test_dataframe['Cleaned_Data'] = test_dataframe['Text'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))

#Removal of punctuations :

test_dataframe['Cleaned_Data'] = test_dataframe['Cleaned_Data'].str.replace('[^\w\s]','')

test_dataframe['Cleaned_Data'] = test_dataframe['Cleaned_Data'].apply(lambda x : x.lower())

#Removing stopwords by using stopwordslist
stop = stopwords.words('english')
test_dataframe['Cleaned_Data'] = test_dataframe['Cleaned_Data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))


from textblob import TextBlob
nltk.download('punkt')
test_dataframe['Cleaned_Data'] = test_dataframe['Cleaned_Data'].apply(lambda x: TextBlob(x).words)

from textblob import Word
test_dataframe['Cleaned_Data'] = test_dataframe['Cleaned_Data'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
test_tfidf_vect1 = TfidfVectorizer(analyzer = 'word', vocabulary = train_tfidf_vect.vocabulary_)
test_tfidf_vect1.fit(test_dataframe['Cleaned_Data'])
x_test_data = test_tfidf_vect1.transform(test_dataframe['Cleaned_Data'])
y_test_data = test_dataframe['Sentiment']

In [ ]:

#Multinomial-Model
multinomial_model = MultinomialNB()
multinomial_model.fit(x_train_data, y_train_data)
validation_predicted_MNB = multinomial_model.predict(x_validation_data)
validation_accuracy_MNB = round(accuracy_score(validation_predicted_MNB,y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_MNB)
print(classification_report(y_validation_data, validation_predicted_MNB))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(multinomial_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_MNB = multinomial_model.predict(x_test_data)
test_accuracy_MNB = round(accuracy_score(test_predicted_MNB, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_MNB)
print(classification_report(y_test_data, test_predicted_MNB))

Validation Data
Accuracy -  77.82
              precision    recall  f1-score   support

           0       0.82      0.68      0.75       661
           1       0.75      0.86      0.80       723

    accuracy                           0.78      1384
   macro avg       0.79      0.77      0.77      1384
weighted avg       0.78      0.78      0.78      1384

Corss Validation Score -  72.105
Testing Data
Accuracy -  79.24
              precision    recall  f1-score   support

           0       0.87      0.69      0.77       912
           1       0.74      0.89      0.81       909

    accuracy                           0.79      1821
   macro avg       0.80      0.79      0.79      1821
weighted avg       0.80      0.79      0.79      1821



In [ ]:
#svm-Model

svm_model = LinearSVC()
svm_model.fit(x_train_data, y_train_data)
validation_predicted_svm = svm_model.predict(x_validation_data)
validation_accuracy_svm = round(accuracy_score(validation_predicted_svm,y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_svm)
print(classification_report(y_validation_data, validation_predicted_svm))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(svm_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_svm = svm_model.predict(x_test_data)
test_accuracy_svm = round(accuracy_score(test_predicted_svm, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_svm)
print(classification_report(y_test_data, test_predicted_svm))

Validation Data
Accuracy -  76.81
              precision    recall  f1-score   support

           0       0.77      0.73      0.75       661
           1       0.77      0.80      0.78       723

    accuracy                           0.77      1384
   macro avg       0.77      0.77      0.77      1384
weighted avg       0.77      0.77      0.77      1384

Corss Validation Score -  70.568
Testing Data
Accuracy -  78.8
              precision    recall  f1-score   support

           0       0.81      0.76      0.78       912
           1       0.77      0.82      0.79       909

    accuracy                           0.79      1821
   macro avg       0.79      0.79      0.79      1821
weighted avg       0.79      0.79      0.79      1821



In [ ]:
#KNN-Model

knn_model = KNeighborsClassifier(n_neighbors = 10)
knn_model.fit(x_train_data, y_train_data)
validation_predicted_KNN = knn_model.predict(x_validation_data)
validation_accuracy_KNN = round(accuracy_score(validation_predicted_KNN, y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_KNN)
print(classification_report(y_validation_data, validation_predicted_KNN))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(knn_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_KNN = knn_model.predict(x_test_data)
test_accuracy_KNN = round(accuracy_score(test_predicted_KNN, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_KNN)
print(classification_report(y_test_data, test_predicted_KNN))

Validation Data
Accuracy -  66.18
              precision    recall  f1-score   support

           0       0.62      0.78      0.69       661
           1       0.73      0.55      0.63       723

    accuracy                           0.66      1384
   macro avg       0.67      0.67      0.66      1384
weighted avg       0.68      0.66      0.66      1384

Corss Validation Score -  54.528
Testing Data
Accuracy -  68.7
              precision    recall  f1-score   support

           0       0.65      0.80      0.72       912
           1       0.74      0.57      0.65       909

    accuracy                           0.69      1821
   macro avg       0.70      0.69      0.68      1821
weighted avg       0.70      0.69      0.68      1821



In [ ]:

#DecisionTree

dt_model = DecisionTreeClassifier()
dt_model.fit(x_train_data, y_train_data)
validation_predicted_DT = dt_model.predict(x_validation_data)
validation_accuracy_DT = round(accuracy_score(validation_predicted_DT, y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_DT)
print(classification_report(y_validation_data, validation_predicted_DT))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(dt_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_DT = dt_model.predict(x_test_data)
test_accuracy_DT = round(accuracy_score(test_predicted_DT, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_DT)
print(classification_report(y_test_data, test_predicted_DT))

Validation Data
Accuracy -  65.68
              precision    recall  f1-score   support

           0       0.64      0.66      0.65       661
           1       0.68      0.66      0.67       723

    accuracy                           0.66      1384
   macro avg       0.66      0.66      0.66      1384
weighted avg       0.66      0.66      0.66      1384

Corss Validation Score -  62.768
Testing Data
Accuracy -  66.45
              precision    recall  f1-score   support

           0       0.66      0.67      0.67       912
           1       0.66      0.66      0.66       909

    accuracy                           0.66      1821
   macro avg       0.66      0.66      0.66      1821
weighted avg       0.66      0.66      0.66      1821



In [ ]:
#RandomForest- Classifier

rf_model = RandomForestClassifier()
rf_model.fit(x_train_data, y_train_data)
validation_predicted_RF = rf_model.predict(x_validation_data)
validation_accuracy_RF = round(accuracy_score(validation_predicted_RF, y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_RF)
print(classification_report(y_validation_data, validation_predicted_RF))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(rf_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_RF = rf_model.predict(x_test_data)
test_accuracy_RF = round(accuracy_score(test_predicted_RF, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_RF)
print(classification_report(y_test_data, test_predicted_RF))

Validation Data
Accuracy -  72.18
              precision    recall  f1-score   support

           0       0.72      0.67      0.70       661
           1       0.72      0.77      0.74       723

    accuracy                           0.72      1384
   macro avg       0.72      0.72      0.72      1384
weighted avg       0.72      0.72      0.72      1384

Corss Validation Score -  66.062
Testing Data
Accuracy -  74.35000000000001
              precision    recall  f1-score   support

           0       0.75      0.72      0.74       912
           1       0.73      0.76      0.75       909

    accuracy                           0.74      1821
   macro avg       0.74      0.74      0.74      1821
weighted avg       0.74      0.74      0.74      1821



In [ ]:
#XGB-Classifier

xgb_model = XGBClassifier()
xgb_model.fit(x_train_data, y_train_data)
validation_predicted_xgb = xgb_model.predict(x_validation_data)
validation_accuracy_xgb = round(accuracy_score(validation_predicted_xgb, y_validation_data),4)*100
print("Validation Data")
print("Accuracy - ", validation_accuracy_xgb)
print(classification_report(y_validation_data, validation_predicted_xgb))
kfold = KFold(10, random_state = 7, shuffle=True)
scoring = 'accuracy'
print("Corss Validation Score - ", round(cross_val_score(xgb_model, x_test_data, y_test_data, cv=kfold).mean()*100,3))
test_predicted_xgb = xgb_model.predict(x_test_data)
test_accuracy_xgb = round(accuracy_score(test_predicted_xgb, y_test_data),4)*100
print("Testing Data")
print("Accuracy - ", test_accuracy_xgb)
print(classification_report(y_test_data, test_predicted_xgb))

Validation Data
Accuracy -  63.800000000000004
              precision    recall  f1-score   support

           0       0.71      0.41      0.52       661
           1       0.61      0.85      0.71       723

    accuracy                           0.64      1384
   macro avg       0.66      0.63      0.61      1384
weighted avg       0.66      0.64      0.62      1384

Corss Validation Score -  62.052
Testing Data
Accuracy -  64.25
              precision    recall  f1-score   support

           0       0.75      0.43      0.55       912
           1       0.60      0.85      0.70       909

    accuracy                           0.64      1821
   macro avg       0.67      0.64      0.63      1821
weighted avg       0.67      0.64      0.63      1821



In [ ]:
Accuracies:

MultinomialNB - 72.10%
SVM - 78.8%
KNN - 68.7%
Decision Tree - 66.45%
Random Forest - 74.35%
XG Boost - 64.25%